Preparando o dataset novamente 



In [2]:
from PIL import Image
import numpy as np
from numpy import array
from tkinter import _flatten
import math
import matplotlib.pyplot as plt
import os


In [3]:
import numpy


gender = [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1]

final = []
for i in gender:
  if i == 0:
    for j in range(0,10):
      final.append(0)
    #new_list = [0,0,0,0,0,0,0,0,0,0]
  else:
    for j in range(0,10):
      final.append(1)


 
gender = np.asarray(final)
gender = np.expand_dims(gender, axis=1)
gender.shape




(400, 1)

In [4]:
from PIL import Image


folder = './images/'  # Updated folder path

def load_images_from_folder(folder):
  i = 1
  for filename in os.listdir(folder):
    im = Image.open(os.path.join(folder,filename))
    data = np.array(im)
    flattened = data.flatten()
    reshape_fla = np.expand_dims(flattened, axis=0)
    if i == 1:
      final = reshape_fla
      i = 2
    else:
      final = np.vstack((final,reshape_fla))

  return final


pixels = load_images_from_folder(folder)

pixels.shape


(400, 5600)

In [1]:
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score

In [5]:
y = gender
X = pixels

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)



In [7]:
# Treinando o modelo decision_tree
dec_tree_def_clf = DecisionTreeClassifier()
dec_tree_def_clf.fit(X_train, y_train)

# Predizendo saídas para o dataset de treinamento
y_train_pred = dec_tree_def_clf.predict(X_train)

# Predizendo saídas para o dataset de testes
y_test_pred = dec_tree_def_clf.predict(X_test)

# Calculando AUC em treinamento e em teste
roc_auc_train = roc_auc_score(y_train, y_train_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print(f"AUC_train = {roc_auc_train}, AUC_tes = {roc_auc_test}")

AUC_train = 1.0, AUC_tes = 0.6285714285714287


Decision Tree Classifier com hiperparâmetros otimizados via grid-search

In [8]:
from sklearn.model_selection import GridSearchCV

# Definindo o grid de parâmetros a serem testados
param_grid = {
  'criterion': ['gini', 'entropy'],
  'max_depth': [None, 5, 10, 15],
  'min_samples_split': [2, 5, 10],
  'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
  estimator = DecisionTreeClassifier(),
  param_grid = param_grid,
  cv = 5
)

# Ajustando o GridSearch ao conjunto de treinamento
grid_search.fit(X_train, y_train)

# Obtendo os melhores parâmetros encontrados
dec_tree_gs_best_params = grid_search.best_params_
print(dec_tree_gs_best_params)

{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10}


In [9]:
# Treinando o modelo
dec_tree_gs_clf = DecisionTreeClassifier(**dec_tree_gs_best_params)
dec_tree_gs_clf.fit(X_train, y_train)

# Predizendo saídas para o dataset de treinamento
y_train_pred = dec_tree_gs_clf.predict(X_train)

# Predizendo saídas para o dataset de testes
y_test_pred = dec_tree_gs_clf.predict(X_test)

# Calculando AUC em treinamento e em teste
roc_auc_train = roc_auc_score(y_train, y_train_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print(f"AUC_train = {roc_auc_train}, AUC_tes = {roc_auc_test}")

AUC_train = 0.8982758620689656, AUC_tes = 0.657142857142857


Decision Tree Classifier com hiperparâmetros otimizados via random-search

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Definindo o espaço de busca para os hiperparâmetros
param_dist = {
  'criterion': ['gini', 'entropy'],
  'max_depth': randint(1, 20),
  'min_samples_split': randint(2, 20),
  'min_samples_leaf': randint(1, 10)
}

random_search = RandomizedSearchCV(
  estimator = DecisionTreeClassifier(),
  param_distributions = param_dist,
  n_iter = 200,
  cv = 5
)

# Ajustando a pesquisa aleatória ao conjunto de treinamento
random_search.fit(X_train, y_train)

# Obtendo os melhores parâmetros encontrados
dec_tree_rs_best_params = random_search.best_params_
print(dec_tree_rs_best_params)

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 6, 'min_samples_split': 11}


In [11]:
# Treinando o modelo
dec_tree_rs_clf = DecisionTreeClassifier(**dec_tree_rs_best_params)
dec_tree_rs_clf.fit(X_train, y_train)

# Predizendo saídas para o dataset de treinamento
y_train_pred = dec_tree_rs_clf.predict(X_train)

# Predizendo saídas para o dataset de testes
y_test_pred = dec_tree_rs_clf.predict(X_test)

# Calculando AUC em treinamento e em teste
roc_auc_train = roc_auc_score(y_train, y_train_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print(f"AUC_train = {roc_auc_train}, AUC_tes = {roc_auc_test}")

AUC_train = 0.7229885057471264, AUC_tes = 0.5785714285714285


Random Forest Classifier com hiperparâmetros otimizados via random-search

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Definindo o espaço de busca para os hiperparâmetros
param_dist = {
    'n_estimators': randint(10, 200),
    'criterion': ['gini', 'entropy'],
    'max_depth': [None] + list(randint(1, 20).rvs(5)),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}

random_search = RandomizedSearchCV(
  estimator=RandomForestClassifier(),
  param_distributions=param_dist,
  n_iter=200,
  cv=5,
  random_state=42
)

# Ajustando a pesquisa aleatória ao conjunto de treinamento
random_search.fit(X_train, y_train)

# Obtendo os melhores parâmetros encontrados
rforest_rs_best_params = random_search.best_params_
print(rforest_rs_best_params)

C:\Users\Drac3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\Drac3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\Drac3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y 

KeyboardInterrupt: 

In [ ]:
# Treinando o modelo
rforest_rs_clf = RandomForestClassifier(**rforest_rs_best_params)
rforest_rs_clf.fit(X_train, y_train)

# Predizendo saídas para o dataset de treinamento
y_train_pred = rforest_rs_clf.predict(X_train)

# Predizendo saídas para o dataset de testes
y_test_pred = rforest_rs_clf.predict(X_test)

# Calculando AUC em treinamento e em teste
roc_auc_train = roc_auc_score(y_train, y_train_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print(f"AUC_train = {roc_auc_train}, AUC_tes = {roc_auc_test}")

KNN com hiperparâmetros otimizados via random-search

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.neighbors import KNeighborsClassifier

# Definindo o espaço de busca para os hiperparâmetros
param_dist = {
    'n_neighbors': randint(1, 50),
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # para a distância de Minkowski (1 para Manhattan, 2 para Euclidiana)
}

random_search = RandomizedSearchCV(
  estimator = KNeighborsClassifier(),
  param_distributions = param_dist,
  n_iter = 200,
  cv = 5
)

# Ajustando a pesquisa aleatória ao conjunto de treinamento
random_search.fit(X_train, y_train)

# Obtendo os melhores parâmetros encontrados
knn_rs_best_params = random_search.best_params_
print(knn_rs_best_params)

C:\Users\Drac3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Drac3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Drac3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

{'n_neighbors': 4, 'p': 2, 'weights': 'distance'}


C:\Users\Drac3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Drac3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Drac3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

In [14]:
# Treinando o modelo
knn_rs_clf = KNeighborsClassifier(**knn_rs_best_params)
knn_rs_clf.fit(X_train, y_train)

# Predizendo saídas para o dataset de treinamento
y_train_pred = knn_rs_clf.predict(X_train)

# Predizendo saídas para o dataset de testes
y_test_pred = knn_rs_clf.predict(X_test)

# Calculando AUC em treinamento e em teste
roc_auc_train = roc_auc_score(y_train, y_train_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print(f"AUC_train = {roc_auc_train}, AUC_tes = {roc_auc_test}")

AUC_train = 1.0, AUC_tes = 0.7428571428571429


C:\Users\Drac3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Cross-validation com Pipelines



In [20]:


from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer


# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando pipeline para pré-processamento e classificação
pipeline = Pipeline([
    ('classifier', DecisionTreeClassifier())
])

# Treinando o modelo e avaliando usando cross-validation
scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')

# Ajustando o modelo final usando todos os dados de treinamento
pipeline.fit(X_train, y_train)

# Avaliando o modelo no conjunto de teste
roc_auc_test = roc_auc_score(y_test, pipeline.predict(X_test))

print(f"AUC_train (CV) = {scores.mean()}, AUC_test = {roc_auc_test}")

AUC_train (CV) = 0.85625, AUC_test = 0.6799687010954616


Cross-validation com Pipelines incluindo random search

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.metrics import roc_auc_score
import pandas as pd


# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando pipeline para pré-processamento e classificação
pipeline = Pipeline([
    ('classifier', DecisionTreeClassifier())
])

# Definindo o espaço de busca de hiperparâmetros
param_dist = {
    'classifier__max_depth': randint(1, 20),  # Profundidade máxima da árvore
    'classifier__min_samples_split': randint(2, 20),  # Número mínimo de amostras necessárias para dividir um nó interno
    'classifier__min_samples_leaf': randint(1, 20),  # Número mínimo de amostras necessárias para estar em um nó folha
}

# Criando o objeto RandomizedSearchCV
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=50, cv=5, scoring='accuracy')

# Executando a busca aleatória de hiperparâmetros
random_search.fit(X_train, y_train)

# Melhores hiperparâmetros encontrados
best_params = random_search.best_params_

# Melhor estimador encontrado
best_estimator = random_search.best_estimator_

# Avaliando o modelo no conjunto de teste usando os melhores hiperparâmetros encontrados
roc_auc_test = roc_auc_score(y_test, best_estimator.predict(X_test))

print(f"Best parameters: {best_params}")
print(f"AUC_train (CV) = {random_search.best_score_}, AUC_test = {roc_auc_test}")


Best parameters: {'classifier__max_depth': 11, 'classifier__min_samples_leaf': 19, 'classifier__min_samples_split': 18}
AUC_train (CV) = 0.890625, AUC_test = 0.47183098591549294
